In [ ]:
class CKYParser:
    def __init__(self, grammar):
        self.grammar = grammar
        self.grammar_dict = self.create_grammar_dict()

    def create_grammar_dict(self):
        grammar_dict = {}
        for rule in self.grammar:
            for lhs, rhs in rule.items():
                if lhs not in grammar_dict:
                    grammar_dict[lhs] = []
                grammar_dict[lhs].append(rhs)
        return grammar_dict

    def parse(self, sentence):
        n = len(sentence)
        parsing_table = [[set() for _ in range(n)] for _ in range(n)]
        backtrack_table = [[{} for _ in range(n)] for _ in range(n)]

        for j in range(n):
            word = sentence[j]
            for lhs, productions in self.grammar_dict.items():
                for rhs in productions:
                    if isinstance(rhs, str) and rhs == word:
                        parsing_table[j][j].add(lhs)
                        backtrack_table[j][j][lhs] = word

        for length in range(2, n + 1):
            for i in range(n - length + 1):
                j = i + length - 1
                for k in range(i, j):
                    for lhs, productions in self.grammar_dict.items():
                        for rhs in productions:
                            if isinstance(rhs, tuple) and len(rhs) == 2:
                                A, B = rhs
                                if A in parsing_table[i][k] and B in parsing_table[k + 1][j]:
                                    parsing_table[i][j].add(lhs)
                                    backtrack_table[i][j][lhs] = (A, B, i, k, k + 1, j)

        if 'S' in parsing_table[0][n - 1]:
            return self.build_tree(0, n - 1, 'S', backtrack_table, sentence)
        else:
            return None

    def build_tree(self, i, j, symbol, backtrack_table, sentence):
        if i == j:
            return sentence[i]
        else:
            A, B, i1, j1, k1, j2 = backtrack_table[i][j][symbol]
            left_subtree = self.build_tree(i, j1, A, backtrack_table, sentence)
            right_subtree = self.build_tree(k1, j2, B, backtrack_table, sentence)
            return (symbol, left_subtree, right_subtree)

In [ ]:
def print_tree(tree, indent=""):
    if isinstance(tree, tuple):
        print(indent + str(tree[0]))
        print_tree(tree[1], indent + "  ")
        print_tree(tree[2], indent + "  ")
    else:
        print(indent + tree)

In [ ]:
grammar = [
    {"S": ("NP", "VP")},
    {"NP": ("Det", "N")},
    {"VP": ("V", "NP")},
    {"Det": "the"},
    {"Det": "a"},
    {"N": "cat"},
    {"N": "dog"},
    {"V": "chases"},
    {"V": "sees"}
]

sentence = input("Enter a sentence as per the grammar: ").lower().split()

Enter a sentence as per the grammar: The dog chases the cat


In [ ]:
parser = CKYParser(grammar)
parse_tree = parser.parse(sentence)

if parse_tree:
    print("Parse Tree:")
    print_tree(parse_tree)
else:
    print("No valid parse tree found.")

Parse Tree:
S
  NP
    the
    dog
  VP
    chases
    NP
      the
      cat
